In [1]:
# Code to Fig. 2 of Sauthoff and others, 2024
# This code requires a XX GB server or local memory
#
# Written 2023-11-11 by W. Sauthoff (wsauthoff.github.io)

In [2]:
# Import libraries
import datetime
# import earthaccess
import geopandas as gpd
# from IPython.display import clear_output
import matplotlib
# import matplotlib.cm as cm
from matplotlib.collections import LineCollection
import matplotlib.colors as colors
import matplotlib.dates as mdates
# from matplotlib.patches import Rectangle
# from matplotlib.legend_handler import HandlerTuple
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
# from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import numpy as np
import os
# from os import path
import pandas as pd
# from pyproj import CRS, Transformer
import rioxarray
# from rioxarray.exceptions import NoDataInBounds
# from shapely.geometry import box, Polygon
from shapely.ops import unary_union
# from skimage import measure
# import xarray as xr

# from IPython.display import Audio, display
# def play_sound():
#     display(Audio(url="http://codeskulptor-demos.commondatastorage.googleapis.com/pang/pop.mp3", autoplay=True))

# Define data directories dependent on home environment
# Replace with your directory file paths
if os.getenv('HOME') == '/home/jovyan':
    DATA_DIR = '/home/jovyan/data'
    SCRIPT_DIR = '/home/jovyan/repos_my/script_dir'
    OUTPUT_DIR = '/home/jovyan/1_outlines_candidates/output/Fig2_lake_reexamination.ipynb'

# # Define constants and coordinate transforms for the geodesic area calculation
# CRS_LL = "EPSG:4326" # wgs84 in lon,lat
# GEOD = CRS(CRS_LL).get_geod() # geod object for calculating geodesic area on defined ellipsoid
# CRS_XY = "EPSG:3031" # Antarctic Polar Stereographic in x, y
# XY_TO_LL = Transformer.from_crs(CRS_XY, CRS_LL, always_xy = True) # make coord transformer

# # Change default font to increase font size
# plt.rcParams.update({'font.size': 8})

# Functions

In [3]:
def timestamp_to_fractional_year(timestamp):
    # Check if the year is a leap year
    year = timestamp.year
    if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
        days_in_year = 366
    else:
        days_in_year = 365

    # Calculate the day of the year
    day_of_year = timestamp.timetuple().tm_yday

    # Calculate the fractional year
    fractional_year = year + (day_of_year - 1) / days_in_year
    
    return fractional_year

# # Example usage
# timestamp = pd.Timestamp('2023-03-07 12:34:56')
# fractional_year = timestamp_to_fractional_year(timestamp)
# print(f"Fractional Year: {fractional_year}")

# Import datasets

In [4]:
# Import subglacial lake outlines 
S09_outlines = gpd.read_file('/home/jovyan/1_outlines_candidates/Sauthoff-2024-J.Glaciol./output/lake_outlines/static_outlines/S09_outlines.geojson')
SF18_outlines = gpd.read_file('/home/jovyan/1_outlines_candidates/Sauthoff-2024-J.Glaciol./output/lake_outlines/static_outlines/SF18_outlines.geojson')
SF18_outlines_SF18only = gpd.read_file('/home/jovyan/1_outlines_candidates/Sauthoff-2024-J.Glaciol./output/lake_outlines/static_outlines/SF18_outlines_SF18only.geojson')
lakes_gdf = gpd.read_file('/home/jovyan/1_outlines_candidates/Sauthoff-2024-J.Glaciol./output/lake_outlines/static_outlines/lakes_gdf.geojson')
lakes_gdf_postSF18 = gpd.read_file('/home/jovyan/1_outlines_candidates/Sauthoff-2024-J.Glaciol./output/lake_outlines/static_outlines/lakes_gdf_postSF18.geojson')

In [5]:
# Import MODIS Mosaic of Antarctica (MOA) surface imagery
# https://nsidc.org/data/nsidc-0730/versions/1
# Relocate to data_dir
# Open into an xarray.DataArray
# moa_lowres = DATA_DIR + '/surface_imagery/MODIS_MOA/2014/moa750_2014_hp1_v01.tif' 
# moa_lowres_da = rioxarray.open_rasterio(moa_lowres)

moa_highres = DATA_DIR + '/surface_imagery/MODIS_MOA/2014/moa125_2014_hp1_v01.tif' 
moa_highres_da = rioxarray.open_rasterio(moa_highres)

Warning 1: TIFFReadDirectory:Invalid data type for tag StripByteCounts
Warning 1: TIFFFetchNormalTag:Incorrect value for "GeoKeyDirectory"; tag ignored
Warning 1: TIFFFetchNormalTag:ASCII value for tag "GeoASCIIParams" contains null byte in value; value incorrectly truncated during reading due to implementation limitations


In [6]:
cyc_dates = pd.read_csv('output/cycle_dates.csv', parse_dates=['cyc_start_dates', 'midcyc_dates', 'cyc_end_dates'])

# Fig. 2

In [ ]:
fig, ax = plt.subplots(3,4, figsize=(15,12))

# Define colors and linestyles that will be reused and create lines for legend
S09_color = 'paleturquoise'
SF18_color  = 'turquoise'
# lake_locations_postSF18_color = 'darkturquoise'
S09_linestyle=(0, (1, 2))
SF18_linestyle=(0, (1, 1))
S09_line = plt.Line2D((0, 1), (0, 0), color=S09_color, linestyle=S09_linestyle, linewidth=2)
SF18_line = plt.Line2D((0, 1), (0, 0), color=SF18_color, linestyle=SF18_linestyle, linewidth=2)

# Panel A - evolving outlines ------------------------------------------------------
# Plot static and evolving outlines onto MOA surface imagery
# Open static outline(s) and evolving outlines and geometric calculation comparison geodataframes for plotting
S09_lake_gdf = S09_outlines[S09_outlines['Name'] == 'Whillans_4']
SF18_lake_gdf = SF18_outlines[SF18_outlines['name'] == 'ConwaySubglacialLake']
evolving_outlines_gdf = gpd.read_file(os.path.join(
    os.getcwd(), 'output/lake_outlines/evolving_outlines/{}.geojson'.format(SF18_lake_gdf['name'].values[0])))
geom_calcs_df = pd.read_csv(os.path.join(
    os.getcwd(), 'output/lake_outlines/compare_evolving_static_outlines/{}.csv'.format(SF18_lake_gdf['name'].values[0])))
# Convert of strings to datetime
geom_calcs_df['midcyc_datetime'] = pd.to_datetime(geom_calcs_df['midcyc_datetime'])

# Combine static outline(s) with evolving outlines in unary union to plot all within bounds of plot
all_outlines_unary_union = unary_union([S09_lake_gdf.geometry.iloc[0], SF18_lake_gdf.geometry.iloc[0]] + list(evolving_outlines_gdf.geometry))
x_min, y_min, x_max, y_max = all_outlines_unary_union.bounds
buffer_frac = 0.2
x_buffer = abs(x_max-x_min)*buffer_frac
y_buffer = abs(y_max-y_min)*buffer_frac
mask_x = (moa_highres_da.x >= x_min-x_buffer) & (moa_highres_da.x <= x_max+x_buffer)
mask_y = (moa_highres_da.y >= y_min-y_buffer) & (moa_highres_da.y <= y_max+y_buffer)
moa_highres_da_subset = moa_highres_da.where(mask_x & mask_y, drop=True)
ax[0,0].imshow(moa_highres_da_subset[0,:,:], cmap="gray", clim=[14000, 17000], extent=[x_min-x_buffer, x_max+x_buffer, y_min-y_buffer, y_max+y_buffer])

# Pick colormap and make continuous cmap discrete for evolving outlines
colormap = 'plasma'
continuous_cmap = matplotlib.colormaps[colormap]
discrete_cmap = colors.ListedColormap(continuous_cmap(np.linspace(0, 1, len(cyc_dates['midcyc_dates'])-1)))

# Norm to time variable
norm = plt.Normalize(mdates.date2num(cyc_dates['midcyc_dates'].iloc[0]), 
                     mdates.date2num(cyc_dates['midcyc_dates'].iloc[-1]))

# Use for loop to store each time slice as line segment to use in legend
# And plot each outline in the geopandas geodataframe and color by date
lines = []  # list of lines to be used for the legend
for idx, dt in enumerate(cyc_dates['midcyc_dates']):
    x = 1; y = 1
    line, = ax[0,0].plot(x, y, color=discrete_cmap(norm(mdates.date2num(cyc_dates['midcyc_dates'][idx]))), linewidth=3)
    lines.append(line)
    
    # Filter rows that match the current time slice
    evolving_outlines_gdf_dt_sub = evolving_outlines_gdf[evolving_outlines_gdf['midcyc_datetime'] == dt]

    # Plotting the subset if not empty
    if not evolving_outlines_gdf_dt_sub.empty:
        evolving_outlines_gdf_dt_sub.boundary.plot(ax=ax[0,0], color=discrete_cmap(norm(mdates.date2num(cyc_dates['midcyc_dates'][idx]))), linewidth=0.75)

# Set axes limit
ax[0,0].set(xlim=(x_min-x_buffer, x_max+x_buffer), ylim=(y_min-y_buffer, y_max+y_buffer))

# Panel - da/dt ---------------------------------------------
ax[0,1].axhline(np.divide(S09_lake_gdf['area (m^2)'], 1e6).values, color=S09_color, linestyle=S09_linestyle, linewidth=3)
ax[0,1].axhline(np.divide(SF18_lake_gdf['area (m^2)'], 1e6).values, color=SF18_color, linestyle=SF18_linestyle, linewidth=3)

# Group by the 'date' column and sum the 'area' column
grouped = evolving_outlines_gdf.groupby('midcyc_datetime')['area (m^2)'].sum().reset_index()
grouped_data = grouped['area (m^2)'].tolist()
grouped_data_dates = grouped['midcyc_datetime'].tolist()

# Plot multi-colored line and scatter for data points
x=[timestamp_to_fractional_year(date) for date in grouped_data_dates]
y=np.divide(grouped_data, 1e6)
# ax[0,1].plot(x,y)
points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)
# Create a continuous norm'ed 
norm = plt.Normalize(np.min(x), np.max(x))  # Normalize to map data points to colors
lc = LineCollection(segments, cmap=colormap, norm=norm, linestyle=(0,(1,1)))
# Set the values used for colormapping
lc.set_array(x)
lc.set_linewidth(2)
line = ax[0,1].add_collection(lc)
ax[0,1].scatter(x, y, c=x, cmap=colormap)

# Specify the years you want as ticks on the x-axis
time_series_even_years = [2012,2014,2016,2018,2020,2022]
# Change x ticks and labels to be even years in time series
ax[0,1].set_xticks(time_series_even_years)
ax[0,1].set_xticklabels([str(year) for year in time_series_even_years])

# Set title
ax[0,1].set_title('wetted area [km$^2$]', size=17.5, pad=8)

# Panel C - dh/dt -------------------------------------------------------
ax[0,2].axhline(0, color='k', linestyle='solid', linewidth=1)

# Plot static outline time series
ax[0,2].plot([timestamp_to_fractional_year(date) for date in geom_calcs_df['midcyc_datetime']], np.cumsum(geom_calcs_df['static_outline_dh_corr (m)']), color=S09_color, linestyle=S09_linestyle, linewidth=3)
# ax[0,2].scatter([timestamp_to_fractional_year(date) for date in geom_calcs_df['midcyc_datetime']], np.cumsum(geom_calcs_df['static_outline_dh_corr (m)']), facecolor=S09_color, linestyle=S09_linestyle, linewidth=3)

# Calc bias and plot
S09_S24_bias = [a_i - b_i for a_i, b_i in 
    zip(np.cumsum(geom_calcs_df['evolving_outlines_dh_corr (m)']), np.cumsum(geom_calcs_df['static_outline_dh_corr (m)']))]
ax[0,2].plot([timestamp_to_fractional_year(date) for date in geom_calcs_df['midcyc_datetime']],
    S09_S24_bias, color='red', linestyle='solid', linewidth=2)

# Plot multi-colored line and scatter for data points
x=[timestamp_to_fractional_year(date) for date in geom_calcs_df['midcyc_datetime']]
# x=geom_calcs_df['midcyc_datetime']
y=np.cumsum(geom_calcs_df['evolving_outlines_dh_corr (m)'])
# x=[geom_calcs_df['midcyc_datetime'].tolist()]
# y=np.cumsum(geom_calcs_df['evolving_outlines_dh (m)'])
points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)
# Create a continuous norm to map from data points to colors
norm = plt.Normalize(x[0], x[-1])
# norm = plt.Normalize(timestamp_to_fractional_year(x.iloc[0]), timestamp_to_fractional_year(x.iloc[-1]))
lc = LineCollection(segments, cmap=colormap, norm=norm, linestyle=(0,(1,1)))
# Set the values used for colormapping
lc.set_array(x)
lc.set_linewidth(2)
line = ax[0,2].add_collection(lc)
ax[0,2].scatter(x, y, c=x, cmap=colormap)

# Change x ticks and labels to be even years in time series
ax[0,2].set_xticks(time_series_even_years)
ax[0,2].set_xticklabels([str(year) for year in time_series_even_years])

# Set title
ax[0,2].set_title('cumulative\nheight change [m]', size=17.5, pad=8)

# Panel D - dv/dt --------------------------------------------------
# Plot horizontal line at zero for reference
ax[0,3].axhline(0, color='k', linestyle='solid', linewidth=1)

# Plot static outline time series
ax[0,3].plot([timestamp_to_fractional_year(date) for date in geom_calcs_df['midcyc_datetime']], 
    np.divide(np.cumsum(geom_calcs_df['static_outline_dvol_corr (m^3)']), 1e9), 
    color=S09_color, linestyle=S09_linestyle, linewidth=3)

# Calc bias and plot
S09_S24_bias = [a_i - b_i for a_i, b_i in 
    zip(np.cumsum(geom_calcs_df['evolving_outlines_dvol_corr (m^3)']), np.cumsum(geom_calcs_df['static_outline_dvol_corr (m^3)']))]
ax[0,3].plot([timestamp_to_fractional_year(date) for date in geom_calcs_df['midcyc_datetime']],
    np.divide(S09_S24_bias, 1e9), color='red', linestyle='solid', linewidth=2)

# Plot multi-colored line and scatter for data points
x=[timestamp_to_fractional_year(date) for date in geom_calcs_df['midcyc_datetime']]
y=np.divide(np.cumsum(geom_calcs_df['evolving_outlines_dvol_corr (m^3)']), 1e9)
points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)
# Create a continuous norm to map from data points to colors
norm = plt.Normalize(x[0], x[-1])
lc = LineCollection(segments, cmap=colormap, norm=norm, linestyle=(0,(1,1)))
# Set the values used for colormapping
lc.set_array(x)
lc.set_linewidth(2)
line = ax[0,3].add_collection(lc)
ax[0,3].scatter(x, y, c=x, cmap=colormap)

# Change x ticks and labels to be even years in time series
ax[0,3].set_xticks(time_series_even_years)
ax[0,3].set_xticklabels([str(year) for year in time_series_even_years])

# Set title
ax[0,3].set_title('cumulative ice volume\ndisplacement [km$^3$]', size=17.5, pad=8)

# 2nd row of plots -----------------------------------------------------------------------

# Plot static and evolving outlines onto MOA surface imagery
# Open static outline(s) and evolving outlines and geometric calculation comparison geodataframes for plotting
S09_lake_gdf = S09_outlines[S09_outlines['Name'] == 'Foundation_5']
SF18_lake_gdf = SF18_outlines[SF18_outlines['name'] == 'Foundation_5']
evolving_outlines_gdf = gpd.read_file(os.path.join(
    os.getcwd(), 'output/lake_outlines/evolving_outlines/{}.geojson'.format(SF18_lake_gdf['name'].values[0])))
geom_calcs_df = pd.read_csv(os.path.join(
    os.getcwd(), 'output/lake_outlines/compare_evolving_static_outlines/{}.csv'.format(SF18_lake_gdf['name'].values[0])))
# Convert of strings to datetime
geom_calcs_df['midcyc_datetime'] = pd.to_datetime(geom_calcs_df['midcyc_datetime'])

# Combine static outline(s) with evolving outlines in unary union to plot all within bounds of plot
all_outlines_unary_union = unary_union([S09_lake_gdf.geometry.iloc[0]] + list(evolving_outlines_gdf.geometry))
                                       
x_min, y_min, x_max, y_max = all_outlines_unary_union.bounds
buffer_frac = 0.2
x_buffer = abs(x_max-x_min)*buffer_frac
y_buffer = abs(y_max-y_min)*buffer_frac
mask_x = (moa_highres_da.x >= x_min-x_buffer) & (moa_highres_da.x <= x_max+x_buffer)
mask_y = (moa_highres_da.y >= y_min-y_buffer) & (moa_highres_da.y <= y_max+y_buffer)
moa_highres_da_subset = moa_highres_da.where(mask_x & mask_y, drop=True)
ax[1,0].imshow(moa_highres_da_subset[0,:,:], cmap="gray", clim=[14000, 17000], extent=[x_min-x_buffer, x_max+x_buffer, y_min-y_buffer, y_max+y_buffer])

# Norm to time variable
norm = plt.Normalize(mdates.date2num(cyc_dates['midcyc_dates'].iloc[0]), 
                     mdates.date2num(cyc_dates['midcyc_dates'].iloc[-1]))

# Use for loop to plot each outline in the geopandas geodataframe and color by date
for idx, dt in enumerate(cyc_dates['midcyc_dates']):
    # Filter rows that match the current time slice
    evolving_outlines_gdf_dt_sub = evolving_outlines_gdf[evolving_outlines_gdf['midcyc_datetime'] == dt]

    # Plotting the subset if not empty
    if not evolving_outlines_gdf_dt_sub.empty:
        evolving_outlines_gdf_dt_sub.boundary.plot(ax=ax[1,0], color=discrete_cmap(norm(mdates.date2num(cyc_dates['midcyc_dates'][idx]))), linewidth=0.75)

# Set axes limit
ax[1,0].set(xlim=(x_min-x_buffer, x_max+x_buffer), ylim=(y_min-y_buffer, y_max+y_buffer))

# Panel - da/dt ---------------------------------------------
ax[1,1].axhline(np.divide(S09_lake_gdf['area (m^2)'].sum(), 1e6), color=S09_color, linestyle=S09_linestyle, linewidth=3)
# ax[1,1].axhline(np.divide(SF18_lake_gdf['area (m^2)'], 1e6), color=SF18_color, linestyle=SF18_linestyle, linewidth=3)

# Group by the 'date' column and sum the 'area' column
grouped = evolving_outlines_gdf.groupby('midcyc_datetime')['area (m^2)'].sum().reset_index()
grouped_data = grouped['area (m^2)'].tolist()
grouped_data_dates = grouped['midcyc_datetime'].tolist()

# Plot multi-colored line and scatter for data points
x=[timestamp_to_fractional_year(date) for date in grouped_data_dates]
y=np.divide(grouped_data, 1e6)
# ax[1,1].plot(x,y)
points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)
# Create a continuous norm'ed 
norm = plt.Normalize(np.min(x), np.max(x))  # Normalize to map data points to colors
lc = LineCollection(segments, cmap=colormap, norm=norm, linestyle=(0,(1,1)))
# Set the values used for colormapping
lc.set_array(x)
lc.set_linewidth(2)
line = ax[1,1].add_collection(lc)
ax[1,1].scatter(x, y, c=x, cmap=colormap)

# Change x ticks and labels to be even years in time series
ax[1,1].set_xticks(time_series_even_years)
ax[1,1].set_xticklabels([str(year) for year in time_series_even_years])

# Panel C - dh/dt -------------------------------------------------------
ax[1,2].axhline(0, color='k', linestyle='solid', linewidth=1)

# Plot static outline time series
ax[1,2].plot([timestamp_to_fractional_year(date) for date in geom_calcs_df['midcyc_datetime']], np.cumsum(geom_calcs_df['static_outline_dh_corr (m)']), color=S09_color, linestyle=S09_linestyle, linewidth=3)
# ax[1,2].scatter([timestamp_to_fractional_year(date) for date in geom_calcs_df['midcyc_datetime']], np.cumsum(geom_calcs_df['static_outline_dh_corr (m)']), facecolor=S09_color, linestyle=S09_linestyle, linewidth=3)

# Calc bias and plot
S09_S24_bias = [a_i - b_i for a_i, b_i in 
    zip(np.cumsum(geom_calcs_df['evolving_outlines_dh_corr (m)']), np.cumsum(geom_calcs_df['static_outline_dh_corr (m)']))]
ax[1,2].plot([timestamp_to_fractional_year(date) for date in geom_calcs_df['midcyc_datetime']],
    S09_S24_bias, color='red', linestyle='solid', linewidth=2)

# Plot multi-colored line and scatter for data points
x=[timestamp_to_fractional_year(date) for date in geom_calcs_df['midcyc_datetime']]
# x=geom_calcs_df['midcyc_datetime']
y=np.cumsum(geom_calcs_df['evolving_outlines_dh_corr (m)'])
# x=[geom_calcs_df['midcyc_datetime'].tolist()]
# y=np.cumsum(geom_calcs_df['evolving_outlines_dh (m)'])
points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)
# Create a continuous norm to map from data points to colors
norm = plt.Normalize(x[0], x[-1])
# norm = plt.Normalize(timestamp_to_fractional_year(x.iloc[0]), timestamp_to_fractional_year(x.iloc[-1]))
lc = LineCollection(segments, cmap=colormap, norm=norm, linestyle=(0,(1,1)))
# Set the values used for colormapping
lc.set_array(x)
lc.set_linewidth(2)
line = ax[1,2].add_collection(lc)
ax[1,2].scatter(x, y, c=x, cmap=colormap)

# Change x ticks and labels to be even years in time series
ax[1,2].set_xticks(time_series_even_years)
ax[1,2].set_xticklabels([str(year) for year in time_series_even_years])

# Panel D - dv/dt --------------------------------------------------
# Plot horizontal line at zero for reference
ax[1,3].axhline(0, color='k', linestyle='solid', linewidth=1)

# Plot static outline time series
ax[1,3].plot([timestamp_to_fractional_year(date) for date in geom_calcs_df['midcyc_datetime']], 
    np.divide(np.cumsum(geom_calcs_df['static_outline_dvol_corr (m^3)']), 1e9), 
    color=S09_color, linestyle=S09_linestyle, linewidth=3)

# Calc bias and plot
S09_S24_bias = [a_i - b_i for a_i, b_i in 
    zip(np.cumsum(geom_calcs_df['evolving_outlines_dvol_corr (m^3)']), np.cumsum(geom_calcs_df['static_outline_dvol_corr (m^3)']))]
ax[1,3].plot([timestamp_to_fractional_year(date) for date in geom_calcs_df['midcyc_datetime']],
    np.divide(S09_S24_bias, 1e9), color='red', linestyle='solid', linewidth=2)

# Plot multi-colored line and scatter for data points
x=[timestamp_to_fractional_year(date) for date in geom_calcs_df['midcyc_datetime']]
y=np.divide(np.cumsum(geom_calcs_df['evolving_outlines_dvol_corr (m^3)']), 1e9)
points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)
# Create a continuous norm to map from data points to colors
norm = plt.Normalize(x[0], x[-1])
lc = LineCollection(segments, cmap=colormap, norm=norm, linestyle=(0,(1,1)))
# Set the values used for colormapping
lc.set_array(x)
lc.set_linewidth(2)
line = ax[1,3].add_collection(lc)
ax[1,3].scatter(x, y, c=x, cmap=colormap)

# Change x ticks and labels to be even years in time series
ax[1,3].set_xticks(time_series_even_years)
ax[1,3].set_xticklabels([])

# 3rd row of plots -----------------------------------------------------------------------

# Plot static and evolving outlines onto MOA surface imagery
# Open static outline(s) and evolving outlines and geometric calculation comparison geodataframes for plotting
S09_lake_gdf = S09_outlines[S09_outlines['Name'].isin(['Slessor_2', 'Slessor_3'])]
SF18_lake_gdf = SF18_outlines[SF18_outlines['name'] == 'Slessor_23']
evolving_outlines_gdf = gpd.read_file(os.path.join(
    os.getcwd(), 'output/lake_outlines/evolving_outlines/{}.geojson'.format(SF18_lake_gdf['name'].values[0])))
geom_calcs_df = pd.read_csv(os.path.join(
    os.getcwd(), 'output/lake_outlines/compare_evolving_static_outlines/{}.csv'.format(SF18_lake_gdf['name'].values[0])))
# Convert of strings to datetime
geom_calcs_df['midcyc_datetime'] = pd.to_datetime(geom_calcs_df['midcyc_datetime'])

# Combine static outline(s) with evolving outlines in unary union to plot all within bounds of plot
all_outlines_unary_union = unary_union([S09_lake_gdf.geometry.iloc[0], SF18_lake_gdf.geometry.iloc[0]] + list(evolving_outlines_gdf.geometry))
x_min, y_min, x_max, y_max = all_outlines_unary_union.bounds
buffer_frac = 0.3
x_buffer = abs(x_max-x_min)*buffer_frac
y_buffer = abs(y_max-y_min)*buffer_frac
mask_x = (moa_highres_da.x >= x_min-x_buffer) & (moa_highres_da.x <= x_max+x_buffer)
mask_y = (moa_highres_da.y >= y_min-y_buffer) & (moa_highres_da.y <= y_max+y_buffer)
moa_highres_da_subset = moa_highres_da.where(mask_x & mask_y, drop=True)
ax[2,0].imshow(moa_highres_da_subset[0,:,:], cmap="gray", clim=[14000, 17000], extent=[x_min-x_buffer, x_max+x_buffer, y_min-y_buffer, y_max+y_buffer])

# Norm to time variable
norm = plt.Normalize(mdates.date2num(cyc_dates['midcyc_dates'].iloc[0]), 
                     mdates.date2num(cyc_dates['midcyc_dates'].iloc[-1]))

# Use for loop to plot each outline in the geopandas geodataframe and color by date
for idx, dt in enumerate(cyc_dates['midcyc_dates']):
    # Filter rows that match the current time slice
    evolving_outlines_gdf_dt_sub = evolving_outlines_gdf[evolving_outlines_gdf['midcyc_datetime'] == dt]

    # Plotting the subset if not empty
    if not evolving_outlines_gdf_dt_sub.empty:
        evolving_outlines_gdf_dt_sub.boundary.plot(ax=ax[2,0], color=discrete_cmap(norm(mdates.date2num(cyc_dates['midcyc_dates'][idx]))), linewidth=0.75)

# Set axes limit
ax[2,0].set(xlim=(x_min-x_buffer, x_max+x_buffer), ylim=(y_min-y_buffer, y_max+y_buffer))

# Panel - da/dt ---------------------------------------------
ax[2,1].axhline(np.divide(S09_lake_gdf['area (m^2)'].sum(), 1e6), color=S09_color, linestyle=S09_linestyle, linewidth=3)
ax[2,1].axhline(np.divide(SF18_lake_gdf['area (m^2)'].sum(), 1e6), color=SF18_color, linestyle=SF18_linestyle, linewidth=3)

# Group by the 'date' column and sum the 'area' column
grouped = evolving_outlines_gdf.groupby('midcyc_datetime')['area (m^2)'].sum().reset_index()
grouped_data = grouped['area (m^2)'].tolist()
grouped_data_dates = grouped['midcyc_datetime'].tolist()

# Plot multi-colored line and scatter for data points
x=[timestamp_to_fractional_year(date) for date in grouped_data_dates]
y=np.divide(grouped_data, 1e6)
# ax[2,1].plot(x,y)
points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)
# Create a continuous norm'ed 
norm = plt.Normalize(np.min(x), np.max(x))  # Normalize to map data points to colors
lc = LineCollection(segments, cmap=colormap, norm=norm, linestyle=(0,(2,1)))
# Set the values used for colormapping
lc.set_array(x)
lc.set_linewidth(2)
line = ax[2,1].add_collection(lc)
ax[2,1].scatter(x, y, c=x, cmap=colormap)

# Change x ticks and labels to be even years in time series
ax[2,1].set_xticks(time_series_even_years)
ax[2,1].set_xticklabels([str(year) for year in time_series_even_years])

# Panel C - dh/dt -------------------------------------------------------
ax[2,2].axhline(0, color='k', linestyle='solid', linewidth=1)

# Plot static outline time series
ax[2,2].plot([timestamp_to_fractional_year(date) for date in geom_calcs_df['midcyc_datetime']], np.cumsum(geom_calcs_df['static_outline_dh_corr (m)']), color=S09_color, linestyle=S09_linestyle, linewidth=3)
# ax[2,2].scatter([timestamp_to_fractional_year(date) for date in geom_calcs_df['midcyc_datetime']], np.cumsum(geom_calcs_df['static_outline_dh_corr (m)']), facecolor=S09_color, linestyle=S09_linestyle, linewidth=3)

# Calc bias and plot
S09_S24_bias = [a_i - b_i for a_i, b_i in 
    zip(np.cumsum(geom_calcs_df['evolving_outlines_dh_corr (m)']), np.cumsum(geom_calcs_df['static_outline_dh_corr (m)']))]
ax[2,2].plot([timestamp_to_fractional_year(date) for date in geom_calcs_df['midcyc_datetime']],
    S09_S24_bias, color='red', linestyle='solid', linewidth=2)

# Plot multi-colored line and scatter for data points
x=[timestamp_to_fractional_year(date) for date in geom_calcs_df['midcyc_datetime']]
# x=geom_calcs_df['midcyc_datetime']
y=np.cumsum(geom_calcs_df['evolving_outlines_dh_corr (m)'])
# x=[geom_calcs_df['midcyc_datetime'].tolist()]
# y=np.cumsum(geom_calcs_df['evolving_outlines_dh (m)'])
points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)
# Create a continuous norm to map from data points to colors
norm = plt.Normalize(x[0], x[-1])
# norm = plt.Normalize(timestamp_to_fractional_year(x.iloc[0]), timestamp_to_fractional_year(x.iloc[-1]))
lc = LineCollection(segments, cmap=colormap, norm=norm, linestyle=(0,(2,1)))
# Set the values used for colormapping
lc.set_array(x)
lc.set_linewidth(2)
line = ax[2,2].add_collection(lc)
ax[2,2].scatter(x, y, c=x, cmap=colormap)

# Change x ticks and labels to be even years in time series
ax[2,2].set_xticks(time_series_even_years)
ax[2,2].set_xticklabels([str(year) for year in time_series_even_years])

# Panel D - dv/dt --------------------------------------------------
# Plot horizontal line at zero for reference
ax[2,3].axhline(0, color='k', linestyle='solid', linewidth=1)

# Plot static outline time series
ax[2,3].plot([timestamp_to_fractional_year(date) for date in geom_calcs_df['midcyc_datetime']], 
    np.divide(np.cumsum(geom_calcs_df['static_outline_dvol_corr (m^3)']), 1e9), 
    color=S09_color, linestyle=S09_linestyle, linewidth=3)

# Calc bias and plot
S09_S24_bias = [a_i - b_i for a_i, b_i in 
    zip(np.cumsum(geom_calcs_df['evolving_outlines_dvol_corr (m^3)']), np.cumsum(geom_calcs_df['static_outline_dvol_corr (m^3)']))]
ax[2,3].plot([timestamp_to_fractional_year(date) for date in geom_calcs_df['midcyc_datetime']],
    np.divide(S09_S24_bias, 1e9), color='red', linestyle='solid', linewidth=2)

# Plot multi-colored line and scatter for data points
x=[timestamp_to_fractional_year(date) for date in geom_calcs_df['midcyc_datetime']]
y=np.divide(np.cumsum(geom_calcs_df['evolving_outlines_dvol_corr (m^3)']), 1e9)
points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)
# Create a continuous norm to map from data points to colors
norm = plt.Normalize(x[0], x[-1])
lc = LineCollection(segments, cmap=colormap, norm=norm, linestyle=(0,(2,1)))
# Set the values used for colormapping
lc.set_array(x)
lc.set_linewidth(2)
line = ax[2,3].add_collection(lc)
ax[2,3].scatter(x, y, c=x, cmap=colormap)

# Change x ticks and labels to be even years in time series
ax[2,3].set_xticks(time_series_even_years)
ax[2,3].set_xticklabels([str(year) for year in time_series_even_years])

# For all plots --------------------------------
# for i in ax: 
#     S09_outlines.boundary.plot(ax=i, edgecolor=S09_color, facecolor='none', linestyle=(0, (1, 2)), linewidth=3, alpha=1, zorder=0)
#     SiegfriedFricker2018_SF18outlines.boundary.plot(ax=i, edgecolor=SF18_color, facecolor='none', linestyle=(0, (1, 1)), linewidth=3, alpha=1, zorder=0)

# # Plot inset map
# axIns = ax.inset_axes([0.01, 0.01, 0.3, 0.3]) # [left, bottom, width, height] (fractional axes coordinates)
# axIns.set_aspect('equal')
# moa_2014_coastline.plot(ax=axIns, color='gray', edgecolor='k', linewidth=0.1, zorder=3)
# moa_2014_groundingline.plot(ax=axIns, color='ghostwhite', edgecolor='k', linewidth=0.1, zorder=3)
# axIns.axis('off')

# # Plot black rectangle to indicate location
# rect = Rectangle((x_min, y_min), (x_max-x_min), (y_max-y_min), fill=False, linewidth=2, color='k', zorder=3)
# axIns.add_artist(rect)

# Label axes
ax[2,0].set_xlabel('x [km]', size=16)
ax[1,0].set_ylabel('y [km]', size=16)

# Create colorbar 
m = plt.cm.ScalarMappable(cmap=discrete_cmap)
m.set_array(np.array([timestamp_to_fractional_year(date) for date in cyc_dates['midcyc_dates']]))
cax = inset_axes(ax[0,0],
                 width="100%",
                 height="3%",
                 loc=3,
                 bbox_to_anchor=[0,1.01,1,1],
                 bbox_transform=ax[0,0].transAxes,
                 borderpad=0,
                 )
cbar=fig.colorbar(m, ticks=np.array([2010,2012,2014,2016,2018,2020,2022]), 
             cax=cax, orientation='horizontal')#.set_label('evolving outline year', size=15)

# Move tick marks and labels to the top
cbar.ax.xaxis.set_ticks_position('top')
cbar.ax.xaxis.set_label_position('top')

# Set the label for the colorbar and adjust its size
cbar.set_label('evolving outline year', size=15, labelpad=10)

for subplot in [ax[0,0], ax[1,0], ax[2,0]]:
    # Plot previous static outline inventories
    S09_outlines.boundary.plot(ax=subplot, edgecolor=S09_color, facecolor='none', linestyle=S09_linestyle, linewidth=3, alpha=1, zorder=0)
    SF18_outlines_SF18only.boundary.plot(ax=subplot, edgecolor=SF18_color, facecolor='none', linestyle=SF18_linestyle, linewidth=3, alpha=1, zorder=0)
    
    # Change polar stereographic m to km
    km_scale = 1e3
    ticks_x = ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x/km_scale))
    subplot.xaxis.set_major_formatter(ticks_x)
    ticks_y = ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x/km_scale))
    subplot.yaxis.set_major_formatter(ticks_y)
    
plt.show()

/srv/conda/envs/notebook/lib/python3.11/site-packages/geopandas/io/file.py:383: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")
/srv/conda/envs/notebook/lib/python3.11/site-packages/geopandas/io/file.py:383: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")
/srv/conda/envs/notebook/lib/python3.11/site-packages/geopandas/io/file.py:383: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")


In [71]:
evolving_outlines_gdf

,area_multiple_search_extent,level,area (m^2),dh (m),vol (m^3),midcyc_datetime,geometry
0,5,0.27,1.138172e+07,0.287461,3.271800e+06,2010-08-17 06:45:00,"POLYGON ((-324730.011 -525005.968, -325735.012..."
1,5,0.27,7.791507e+07,0.307277,2.394155e+07,2011-05-17 23:15:00,"POLYGON ((-317695.004 -515660.929, -317825.906..."
2,5,0.27,1.236404e+07,0.286528,3.542638e+06,2011-05-17 23:15:00,"POLYGON ((-310659.996 -516536.256, -311664.997..."
3,5,0.27,2.427931e+08,0.392276,9.524202e+07,2011-08-17 06:45:00,"POLYGON ((-314680.001 -513988.561, -315685.002..."
4,5,0.27,3.322484e+08,0.479101,1.591806e+08,2011-11-16 11:15:00,"POLYGON ((-313674.999 -512465.836, -314680.001..."
...,...,...,...,...,...,...,...
122,5,0.27,1.619008e+06,0.256958,4.160171e+05,2021-05-17 17:15:00,"POLYGON ((-332770.019 -514451.641, -333308.813..."
123,5,0.27,4.622156e+06,0.273173,1.262648e+06,2021-05-17 17:15:00,"POLYGON ((-327745.014 -513059.617, -328750.015..."
124,5,0.27,1.418910e+07,0.497904,7.064817e+06,2022-02-15 15:45:00,"POLYGON ((-306639.992 -507664.695, -307644.993..."
125,5,0.27,2.544310e+07,0.642405,1.634478e+07,2022-05-17 23:15:00,"POLYGON ((-305634.991 -506986.114, -306639.992..."


In [61]:
geom_calcs_df

,midcyc_datetime,evolving_outlines_darea (m^2),evolving_outlines_dh (m),evolving_region_dh (m),evolving_outlines_dh_corr (m),evolving_outlines_dvol_corr (m^3),static_outline_dh (m),static_region_dh (m),static_outline_dh_corr (m),static_outline_dvol_corr (m^3)
0,2013-11-16 11:15:00,2.467941e+08,-1.058894,-0.103752,-0.955142,-2.357234e+08,-0.817244,-0.257699,-0.559545,-1.512170e+08
1,2014-02-15 15:45:00,2.331373e+08,-1.703150,-0.043055,-1.660095,-3.870300e+08,-1.301605,-0.259832,-1.041773,-2.815392e+08
2,2014-05-17 23:15:00,2.418913e+08,-2.319344,0.014558,-2.333902,-5.645507e+08,-1.915343,-0.284937,-1.630406,-4.406175e+08
3,2014-08-17 06:45:00,2.645266e+08,-2.392419,0.029276,-2.421696,-6.406029e+08,-2.139063,-0.287368,-1.851695,-5.004209e+08
4,2014-11-16 11:15:00,2.660990e+08,-1.835088,-0.009261,-1.825827,-4.858506e+08,-1.637374,-0.234203,-1.403171,-3.792072e+08
5,2015-02-15 15:45:00,2.232810e+08,-1.194560,-0.059985,-1.134575,-2.533290e+08,-0.907455,-0.172107,-0.735349,-1.987280e+08
6,2015-05-17 23:15:00,1.302508e+08,-0.564690,-0.084323,-0.480366,-6.256814e+07,-0.265361,-0.092970,-0.172391,-4.658880e+07
7,2015-08-17 06:45:00,1.413708e+08,0.284073,-0.065933,0.350007,4.948073e+07,0.159567,0.006915,0.152652,4.125417e+07
8,2015-11-16 11:15:00,1.571870e+08,0.779375,-0.045512,0.824887,1.296614e+08,0.397875,0.051900,0.345975,9.349981e+07
9,2016-02-15 15:45:00,1.100800e+08,1.310096,-0.006334,1.316430,1.449127e+08,0.465551,0.040972,0.424579,1.147426e+08


In [56]:
mdates.date2num(cyc_dates['midcyc_dates'].iloc[0])

14838.28125

In [55]:
np.min(x)

2010.6246575342466